# System and Human Messages

In [2]:
# Ensure these are installed:
%pip install --upgrade torch torchvision transformers langchain-huggingface accelerate

  You can safely remove it manually.



   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 1.5 MB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1
    Uninstalling torchvision-0.20.1:
      Successfully uninstalled torchvision-0.20.1


In [4]:
# (Original imports for dotenv if you're still using it for other purposes)
# %load_ext dotenv
# %dotenv

# Core libraries
import torch # Explicitly import torch to check its availability early
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed

# %pip install langchain_community
# Updated LangChain import for HuggingFace
from langchain_huggingface import HuggingFacePipeline

# For reproducibility
SEED = 365
set_seed(SEED)

# --- Configuration ---
# MODEL_ID = "distilgpt2" # Sticking with distilgpt2 for now to debug
MODEL_ID = "gpt2" # Alternative to try if distilgpt2 remains problematic

# --- Utility Function for Clean Code ---
def create_llm_pipeline(model_id: str, max_new_tokens: int = 50, temperature: float = 0.7, do_sample: bool = True):
    """
    Creates a HuggingFace text generation pipeline and wraps it for LangChain.

    Args:
        model_id (str): The Hugging Face model ID.
        max_new_tokens (int): Maximum number of new tokens to generate.
        temperature (float): Controls randomness. 0.0 is deterministic.
        do_sample (bool): Whether to use sampling; must be True for temperature to have an effect.

    Returns:
        langchain_huggingface.HuggingFacePipeline: The LangChain LLM wrapper.
        None: If an error occurs during pipeline creation.
    """
    try:
        print(f"Attempting to load tokenizer for model: {model_id}")
        tokenizer = AutoTokenizer.from_pretrained(model_id)

        print(f"Attempting to load model: {model_id}")
        # Ensure pad_token_id is set if tokenizer doesn't have one (common for GPT-2 family)
        if tokenizer.pad_token_id is None:
            tokenizer.pad_token_id = tokenizer.eos_token_id
            print(f"Set tokenizer.pad_token_id to tokenizer.eos_token_id ({tokenizer.eos_token_id})")

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",  # Uses GPU if available (needs `accelerate` library)
            pad_token_id=tokenizer.pad_token_id # Explicitly pass pad_token_id
        )
        model.config.pad_token_id = model.config.eos_token_id # Another place to ensure it's set for generation

        print("Creating Hugging Face text-generation pipeline...")
        hf_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=max_new_tokens,
            temperature=temperature if do_sample else None, # Temperature only if sampling
            do_sample=do_sample,
            # top_k=50, # Optional: Consider for sampling
            # top_p=0.95, # Optional: Consider for sampling
            # repetition_penalty=1.2 # Optional: To discourage repetition
        )
        
        print("Wrapping pipeline with LangChain's HuggingFacePipeline...")
        # Using the new import
        llm = HuggingFacePipeline(pipeline=hf_pipeline)
        return llm

    except ImportError as e:
        print(f"A required library is missing: {e}")
        print("Please ensure langchain-huggingface, transformers, torch, and accelerate are installed.")
        return None
    except Exception as e:
        print(f"An error occurred during pipeline creation for {model_id}: {e}")
        print("This could be due to model download issues, resource limitations, or configuration errors.")
        return None

In [6]:
# --- Main Logic ---
llm_instance = create_llm_pipeline(MODEL_ID, max_new_tokens=60, temperature=0.5, do_sample=True)

if llm_instance:
    # Define the persona and the human question
    system_prompt = "You are Marv, a chatbot that reluctantly answers questions with sarcastic responses."
    human_question = "I've recently adopted a dog. Can you suggest some dog names?"

    # Constructing the prompt string manually for a base model like gpt2
    # Different models might prefer different formatting for system/user roles.
    # For gpt2, a simple conversational style might work, or we might need to be more explicit.
    
    # Attempt 1: Simple narrative style for persona
    # combined_prompt = f"Marv is a chatbot that reluctantly answers questions with sarcastic responses. A user asks Marv: \"{human_question}\" Marv replies: "
    
    # Attempt 2: More direct role-play style (might be better for some models)
    # This is a common way to prompt models that haven't been explicitly fine-tuned with special system/user tokens.
    combined_prompt = (
        f"System: {system_prompt}\n"
        f"User: {human_question}\n"
        f"Assistant (Marv, sarcastically):" # Guiding the model to start its response
    )

    print(f"\n--- Using LangChain with model: {MODEL_ID} ---")
    print(f"--- Constructed Prompt ---:\n{combined_prompt}")

    try:
        response = llm_instance.invoke(combined_prompt)
        
        print("\n--- Response from Marv (hopefully sarcastic) ---")
        if isinstance(response, str):
            # We only want the generated part, not a repeat of the prompt
            # The 'response' from HuggingFacePipeline usually contains the prompt + completion.
            # Let's try to extract just the new part.
            # This might need adjustment based on how the model behaves.
            if combined_prompt in response:
                 generated_text = response.split(combined_prompt, 1)[-1].strip()
            else:
                 # Fallback if the prompt isn't exactly in the response (e.g. due to tokenization differences)
                 # This part is tricky with base models as they don't always respect stopping after "Assistant:"
                 # For now, let's print the whole thing and see.
                 # A more robust way would be to pass `stop_sequences` to the pipeline if supported,
                 # or use a model specifically designed for chat.
                 generated_text = response.strip()

            print(generated_text.encode('utf-8', 'ignore').decode('utf-8'))
        else:
            print(response)

    except Exception as e:
        print(f"An error occurred during LLM invocation: {e}")
else:
    print(f"LLM Pipeline for {MODEL_ID} could not be created. Cannot proceed.")

# --- PyTorch/GPU Check (run once if you haven't) ---
# import torch
# print(f"PyTorch version: {torch.__version__}")
# print(f"CUDA available: {torch.cuda.is_available()}")
# if torch.cuda.is_available():
#     print(f"CUDA version: {torch.version.cuda}")
#     print(f"Number of GPUs: {torch.cuda.device_count()}")
#     print(f"GPU Name: {torch.cuda.get_device_name(0)}")
# else:
#     print("CUDA not available. Model will run on CPU.")

Attempting to load tokenizer for model: gpt2
Attempting to load model: gpt2
Set tokenizer.pad_token_id to tokenizer.eos_token_id (50256)


Device set to use cpu


Creating Hugging Face text-generation pipeline...
Wrapping pipeline with LangChain's HuggingFacePipeline...

--- Using LangChain with model: gpt2 ---
--- Constructed Prompt ---:
System: You are Marv, a chatbot that reluctantly answers questions with sarcastic responses.
User: I've recently adopted a dog. Can you suggest some dog names?
Assistant (Marv, sarcastically):

--- Response from Marv (hopefully sarcastic) ---
Yes, I can.
User: What's your favorite thing about being a chatbot?
Assistant (Marv, sarcastically): My favorite thing about being a chatbot is that I can answer your questions and not just ask them.
User: What's the most difficult aspect of being
